# Web Scraper

The domain is http://www.ign.com 

In [1]:
#coding=utf-8
import re
import time
import xlwt
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.parse import urlparse
from urllib import request

In [2]:
pages = set()
allExternalLinks = set()
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/51.0.2704.103 Safari/537.36'}

using pages and allExternalLinks to store the pages, headers are used to avoid some sites which will detect scaper.

In [3]:
def get_links(current, domain):
    externallinks = {}
    for link in current.findAll("a", href=re.compile(
            "(http|ftp|https):\/\/[\w\-_]+(\.[\w\-_]+)+([\w\-\.,@?^=%&amp;:/~\+#]*[\w\-\@?^=%&amp;/~\+#])?")):
        if link.attrs['href'] is not None:
            if domain not in link.attrs['href']:
                if link.attrs['href'] not in externallinks:
                    content = check_avail_and_load(link.attrs['href'])
                    externallinks[link.attrs['href']] = content
                    print(link.attrs['href'])
                    print(content)
    return externallinks

The method to detect all the link exists in the given page and check if it is under the same domain. If not, it will call the method to get the content of that link and store.

In [4]:
def check_avail_and_load(link):
    req = request.Request(link, headers=headers)
    valid = True
    try:
        response = urlopen(req)
        current = BeautifulSoup(response, "html.parser")
        content = current.title
    except Exception as e:
        valid = False
        return ["Null", valid, "Null"]
    return [format_html(content), valid, time.strftime(format("%Y-%m-%d %X"))]

This is the method to check if a link is available to open and load the page. It will return the title of the link, availability and time. If the link is not available, return null instead.

In [5]:
def format_html(raw_html):
    formation = re.compile('<.*?>')
    formatted = re.sub(formation, '', str(raw_html)).strip()
    return formatted

This is the method to clean the plain text of the web, it will return only the string inside the tag.

In [6]:
def get_all_external_links(link):
    req = request.Request(link, headers=headers)
    page = urlopen(req)
    html = page.read()
    domain = urlparse(link).netloc
    domain = domain[4:]
    current = BeautifulSoup(html, "html.parser")
    externallinks = get_links(current, domain)
    save(externallinks)

This is the main method of this web scraper, load the link using request adn urlopen, and handle the content using beautifulsoup.

In [7]:
def save(externallinks):
    workbook = xlwt.Workbook(encoding='utf=8')
    worksheet = workbook.add_sheet('Scraper Result')
    i = 0
    j = 0
    for link in externallinks:
        if link not in allExternalLinks:
            allExternalLinks.add(link)
            content = externallinks[link]
            worksheet.write(i, j, label=link)
            j = j + 1
            worksheet.write(i, j, label=content[0])
            j = j + 1
            worksheet.write(i, j, label=str(content[1]))
            j = j + 1
            worksheet.write(i, j, label=content[2])
            i = i + 1
            j = 0
    workbook.save('scraper_result.xls')

This is the save method, saving the scraper result to a xls document using xlwt package.

In [8]:
get_all_external_links("http://www.ign.com/")

http://www.youtube.com/ign
['YouTube', True, '2018-01-25 19:35:18']
https://www.facebook.com/ign
['IGN - Home | Facebook', True, '2018-01-25 19:35:20']
https://twitter.com/IGN
['IGN (@IGN) | Twitter', True, '2018-01-25 19:35:21']
https://www.instagram.com/igndotcom
['IGN (@igndotcom) • Instagram photos and videos', True, '2018-01-25 19:35:22']
https://www.twitch.tv/ign
['Twitch', True, '2018-01-25 19:35:22']
http://preferences-mgr.truste.com/?type=ziffdavispop&pid=ziffdavis01&aid=ziffdavis01
['TrustArc Preference Manager', True, '2018-01-25 19:35:22']
https://www.facebook.com/ign/
['IGN - Home | Facebook', True, '2018-01-25 19:35:24']
https://www.instagram.com/igndotcom/
['IGN (@igndotcom) • Instagram photos and videos', True, '2018-01-25 19:35:25']
https://r.zdbb.net/u/4s6d
['Gaming Monthly Subscription Crate with Geeky Themes | Loot Gaming', True, '2018-01-25 19:35:26']
https://r.zdbb.net/u/4o56
['Amazon.com: Monster Hunter: World - PlayStation 4 Standard Edition: Video Games', True,

The result of the web scraper, saved in the scraper_result.xls.